## install requirements for this code which include:
numpy: helps us generate musical waves using python <br>
pyaudio: converts waves to audio <br>
midiutil: allows us to export music as a MIDI file <br>
mingus: stores information about notes, chords, scales and a lot more that we can use to enhance our music


In [1]:
!pip install -r requirements.txt --quiet

## helper methods to generate waves and play sounds 

In [1]:
def note_frequency(note, octave): # returns the frequency associated with a given note in its numerical form
    A4_freq = 440
    A4_note_number = 57
    note_number = note_to_number(note, octave)
    return A4_freq * 2 ** ((note_number - A4_note_number) / 12)

def note_to_number(note, octave): # converts a note to its corresponding number
    note_map = {"C": 0, "C#": 1, "Db": 1, "D": 2, "D#": 3, "Eb": 3, "E": 4, "F": 5, "F#": 6, "Gb": 6, "G": 7, "G#": 8, "Ab": 8, "A": 9, "A#": 10, "Bb": 10, "B": 11}
    if type(note) == str: # if we are given a note as a string like "C"
        return 12 * (octave + 1) + note_map[note]
    else: # if we are given the note as a number 
        return 12 * (octave + 1) + note


def note_to_number(note, octave): # converts a note to its corresponding number
    note_map = {"C": 0, "C#": 1, "Db": 1, "D": 2, "D#": 3, "Eb": 3, "E": 4, "F": 5, "F#": 6, "Gb": 6, "G": 7, "G#": 8, "Ab": 8, "A": 9, "A#": 10, "Bb": 10, "B": 11}
    if type(note) == str: # if we are given a note as a string like "C"
        return 12 * (octave + 1) + note_map[note]
    else: # if we are given the note as a number 
        return 12 * (octave + 1) + note


def sine_wave(frequency, length, rate=44100): # given a frequency and length, returns a sin wave, (rate is the sampling rate detail of sound)
    t = np.linspace(0, length, int(rate * length))
    wave = np.sin(frequency * 2 * np.pi * t)
    return wave

def cos_wave(frequency, length, rate=44100): # given a frequency and length, returns a cos wave 
    t = np.linspace(0, length, int(rate * length))
    wave = np.cos(np.pi/4 + frequency * 2 * np.pi * t)

    return wave

def generate_white_noise(rate=44100, length=0.05, volume=0.5): # generate uniform random noise given a rate, length and volume
    return np.random.uniform(-volume, volume, int(rate * length))

def generate_sharp_drum_sound(rate=44100, length=0.1, sine_freq=150, volume=0.5): # to generate the metronome tick sound
    """Generate a sharper
 drum sound."""
    t = np.linspace(0, length, int(rate * length))
    
    # Sine wave for body of the drum sound
    sine_wave = np.sin(2 * np.pi * sine_freq * t) * volume

    # Quick decay envelope for sharpness
    decay = np.e ** (-15 * t)

    # Add a bit of noise for sharpness
    noise = np.random.uniform(-0.5, 0.5, sine_wave.shape) * decay

    # Combine sine wave, noise, and decay
    drum_sound = (sine_wave + noise) * decay

    return drum_sound

def play_sound(sound, rate=44100, fade_out_duration=1):  # Fade out in the last 1 second
    # Calculate the number of samples for the fade out
    fade_out_samples = int(rate * fade_out_duration)

    # Extend the sound by repeating the last value for the fade-out duration
    extended_sound = np.append(sound, np.full(fade_out_samples, sound[-1]))

    # Create a fade-out envelope (exponential decay)
    fade_out_envelope = np.linspace(1, 0, fade_out_samples)**2

    # Apply the fade-out envelope to the extended part of the sound
    extended_sound[-fade_out_samples:] *= fade_out_envelope

    # Play the sound with fade-out
    p = pyaudio.PyAudio()
    stream = p.open(format=pyaudio.paFloat32, channels=1, rate=rate, output=True)
    stream.write(extended_sound.astype(np.float32).tobytes())
    stream.stop_stream()
    stream.close()
    p.terminate()


# code to produce musical notes with a metronome tick in the background

In [2]:
import numpy as np
import pyaudio

# ... (Include all the previously defined functions like note_frequency, sine_wave, etc.)

def play_notes_and_chords_with_metronome(chords, octaves, lengths, metronome_tick_length=0.05, ticks_per_beat=4):
    """Play a list of chords with corresponding octaves and lengths, along with a metronome."""
    if not (len(chords) == len(octaves) == len(lengths)):
        raise ValueError("The length of chords, octaves, and lengths lists must be the same.")

    # Generate notes stream
    notes_stream = np.array([], dtype=np.float32)
    total_length = 0  # To calculate total length for the metronome stream
    for chord, octave, length in zip(chords, octaves, lengths):
        total_length += length
        if chord:  # If not an empty list (silence)
            chord_wave = np.zeros(int(length * 44100))  # Start with silence
            for note in chord:  # For each note in the chord
                frequency = note_frequency(note, octave)
                note_wave = sine_wave(frequency, length)
                chord_wave += note_wave
            chord_wave /= len(chord)  # Normalize volume of chord
            notes_stream = np.concatenate((notes_stream, chord_wave))
        else:  # For a blank note (silence)
            silence = np.zeros(int(length * 44100))
            notes_stream = np.concatenate((notes_stream, silence))

    # Generate metronome stream
    metronome_stream = np.array([], dtype=np.float32)
    total_metronome_ticks = int(total_length * ticks_per_beat / metronome_tick_length)
    for _ in range(total_metronome_ticks):
        metronome_stream = np.concatenate((metronome_stream, generate_sharp_drum_sound(length=metronome_tick_length)))
        metronome_stream = np.concatenate((metronome_stream, np.zeros(int((1 / ticks_per_beat - metronome_tick_length) * 44100))))

    # Ensure metronome stream is not longer than notes stream
    if len(metronome_stream) > len(notes_stream):
        metronome_stream = metronome_stream[:len(notes_stream)]

    # Mix the metronome and notes streams
    mixed_stream = 0.5*metronome_stream + notes_stream

    # Play the mixed audio
    play_sound(mixed_stream)



## using mingus to generate scales

In [19]:
play_notes_and_chords_with_metronome([['A'], ['C'], ['A'], ['D']], [2,2,2,2], [0.75] * 4)

In [14]:
# from mingus.core import scales, note

# def find_scale(note: str):
#     return scales.Major(note).descending()
    
# # Later
# # TODO: change the below letter, and run the cell.
# # Note: no code is to be written
# print(find_scale('A'))

## changing the genetic algorithm framework to allow convenient application to music
### we make our genome: [note,length, note, length, $\dots$,]
### Note: each note will be a music letter eg: 'C', 'C#', etc. and the length will be a float
### Note: we also have made slight changes to the mutation, generate_genome, generate population functions

In [5]:
from random import choices, randint, randrange, random, sample
from typing import List, Optional, Callable, Tuple, Union

Genome = List[Union[List[int], float]]
Population = List[Genome]
PopulateFunc = Callable[[], Population]
FitnessFunc = Callable[[Genome], int]
SelectionFunc = Callable[[Population, FitnessFunc], Tuple[Genome, Genome]]
CrossoverFunc = Callable[[Genome, Genome], Tuple[Genome, Genome]]
MutationFunc = Callable[[Genome], Genome]
PrinterFunc = Callable[[Population, int, FitnessFunc], None]


number_map = {0 : "C", 1: "C#", 1: "Db", 2: "D", 3: "D#", 3: "Eb", 4: "E", 5: "F", 6: "F#", 6: "Gb", 7:"G", 8: "G#", 8: "Ab", 9: "A", 10: "A#",10: "Bb", 11: "B"}
note_map = {"C": 0, "C#": 1, "Db": 1, "D": 2, "D#": 3, "Eb": 3, "E": 4, "F": 5, "F#": 6, "Gb": 6, "G": 7, "G#": 8, "Ab": 8, "A": 9, "A#": 10, "Bb": 10, "B": 11}

def generate_genome(len_of_tune) -> Genome: # encoded as a list of two lists, list1: notes, list2: lengths
    # lengths are 0.25, 0.5, and 0.75
    note_list = [] # stores notes as LETTERS
    length_list = [] # stores lengths of each note as a FLOATS

    # TODO: populate the note_list and length_list randomly.
    # warning: do not delete the code below it interleaves the notes and lengths and returns the genome
    # hint: use the note_map dictionary
    for i in range(len_of_tune*2):
        note_list.append([number_map[randint(0,11)]])
        length_list.append(randint(1,3)/4)
    
    # TODO 2: incoporate scales to enhance your random melodies.
    # hint: use the find_scales method from the previous block of code
    # Note: start by incoperating a singular scale.
    
    ### DO NOT CHANGE ###
    genome = []
    for i in range(0, 2*len_of_tune):
        if i % 2 == 0:
            genome.append(note_list[i//2])
        else:
            genome.append(length_list[(i-1)//2])
    return genome
    ### DO NOT CHANGE ###
print(generate_genome(10))

def generate_population(size: int, genome_length: int) -> Population:
    population = []
    for i in range(size):
        population.append(generate_genome(genome_length))
    return population


def single_point_crossover(a: Genome, b: Genome) -> Tuple[Genome, Genome]:
    if len(a) != len(b):
        raise ValueError("Genomes a and b must be of same length")

    length = len(a)
    if length < 2:
        return a, b

    p = randint(1, length - 1)
    return a[0:p] + b[p:], b[0:p] + a[p:]

def letter_mutation(genome: Genome, num: int = 1, probability: float = 0.5) -> Genome:
    for i in range(num):
        if random() > probability: # then we perform the mutation
            index = randint(0, len(genome)-1)
            if type(genome[index]) is float:
                genome[index] = 0.25 * randint(1,4)
            else:
                genome[index] = [number_map[randint(0,11)]]
    return genome


def population_fitness(population: Population, fitness_func: FitnessFunc) -> int:
    return sum([fitness_func(genome) for genome in population])


def selection_pair(population: Population, fitness_func: FitnessFunc) -> Population:
    return sample(
        population=generate_weighted_distribution(population, fitness_func),
        k=2
    )



def generate_weighted_distribution(population: Population, fitness_func: FitnessFunc) -> Population:
    result = []

    for gene in population:
        result += [gene] * int(fitness_func(gene)+1)

    return result


def sort_population(population: Population, fitness_func: FitnessFunc) -> Population:
    return sorted(population, key=fitness_func, reverse=True)


def genome_to_string(genome: Genome) -> str:
    return "".join(map(str, genome))


def print_stats(population: Population, generation_id: int, fitness_func: FitnessFunc):
    print("GENERATION %02d" % generation_id)
    print("=============")
    print("Population: [%s]" % ", ".join([genome_to_string(gene) for gene in population]))
    print("Avg. Fitness: %f" % (population_fitness(population, fitness_func) / len(population)))
    sorted_population = sort_population(population, fitness_func)
    print(
        "Best: %s (%f)" % (genome_to_string(sorted_population[0]), fitness_func(sorted_population[0])))
    print("Worst: %s (%f)" % (genome_to_string(sorted_population[-1]),
                              fitness_func(sorted_population[-1])))
    print("")

    return sorted_population[0]


def run_evolution(
        populate_func: PopulateFunc,
        fitness_func: FitnessFunc,
        fitness_limit: int,
        selection_func: SelectionFunc = selection_pair,
        crossover_func: CrossoverFunc = single_point_crossover,
        mutation_func: MutationFunc = letter_mutation,
        generation_limit: int = 100,
        printer: Optional[PrinterFunc] = None) \
        -> Tuple[Population, int]:
    population = populate_func()

    i = 0
    for i in range(generation_limit):
        population = sorted(population, key=lambda genome: fitness_func(genome), reverse=True)

        if printer is not None:
            printer(population, i, fitness_func)

        if fitness_func(population[0]) >= fitness_limit:
            print("reached fitness limit")
            break

        next_generation = population[0:2] # transfer the two fittest beings
        for j in range(int((len(population)) / 2)-1):
            parents = selection_func(population, fitness_func)
            offspring_a, offspring_b = crossover_func(parents[0], parents[1])
            offspring_a = mutation_func(offspring_a)
            offspring_b = mutation_func(offspring_b)
            next_generation += [offspring_a, offspring_b]

        population = next_generation
        print(population)
        print("NEXT GEN")

    return population, i


[['Bb'], 0.5, ['E'], 0.5, ['Eb'], 0.5, ['Gb'], 0.25, ['C'], 0.75, ['Bb'], 0.75, ['F'], 0.75, ['E'], 0.25, ['D'], 0.75, ['Bb'], 0.25]


## implementing the above framework to make our own genetic algorithm

In [6]:
from midiutil import MIDIFile
from typing import Union, List
from mingus.core import chords, notes, scales



def genome_to_list(genome: Genome): # converts the genome two a notes_list, and a lengths list
    # notes_list is a list of letters ('C', 'C#' etc.) and lengths_list is a list of floats
    assert len(genome) % 2 == 0
    #print(genome)
    play_notes, lengths = [], []
    for i in range(len(genome)):
        if i % 2== 0:
            play_notes.append(genome[i])
        if i %2==1:
           lengths.append(genome[i])

    # TODO given a genome in the format: [note1, len1, note2, len2, .....]  convert it into a notes list and a lengths list

    return play_notes, lengths
print(genome_to_list(generate_genome(10)))

def genome_to_music(genome: Genome): # play the music encoded by a genome
    play_notes, lengths = genome_to_list(genome)
    octaves = [3] * len(lengths)
    print(genome)
    play_notes_and_chords_with_metronome(play_notes, octaves, lengths)

# genome_to_music(generate_population_music(5))  # this works

saver = {}
def fitness_music(genome: Genome):
    genome_value = genome_hash(genome)
    if genome_hash(genome) in saver:
        return saver[genome_value]
    genome_to_music(genome)
    rating = input("rate this song from 1-5")
    if int(rating) == 100:
        save_melody(genome)
    saver[genome_value] = int(rating)
    return int(rating)

def genome_hash(genome: Genome): # hashing the genome
    hash_val = 0
    for i in genome:
        if type(i) is float:
            hash_val += i
        else:
            hash_val += sum([note_map[j] for j in i])
    return hash_val


def save_melody(genome: Genome):
    track = 0
    channel = 0
    time = 0  # In beats
    duration = 1  # In beats
    tempo = 120  # In BPM
    volume = 100  # 0-127, as per the MIDI standard

    MyMIDI = MIDIFile(1)  # One track, defaults to format 1 (tempo track is created)
    MyMIDI.addTempo(track, time, tempo)
    
    play_notes, lengths = genome_to_list(genome)
    timer = 0
    for i in range(len(lengths)):
        for j in play_notes[i]:
            MyMIDI.addNote(track, channel, 33 + note_map[j],timer, lengths[i], volume)
        timer += lengths[i]
    with open("GeneticAlgoMusic.mid", "wb") as output_file:
        MyMIDI.writeFile(output_file)
    


        
        
people_in_population = 5 # number of melodies per generation 
len_melody = 5 # number of notes in a melody
fitness_limit = 10 # if the user ever enters a rating of higher than this, the algorithm will stop at the next generation
max_generations = 10 # generations before algorithm automatically terminates



run_evolution(lambda : generate_population(people_in_population, len_melody), 
fitness_music, fitness_music, selection_pair, single_point_crossover, letter_mutation, max_generations)


([['B'], ['B'], ['D'], ['Eb'], ['C'], ['F'], ['B'], ['Eb'], ['B'], ['C']], [0.25, 0.25, 0.75, 0.75, 0.75, 0.25, 0.75, 0.5, 0.5, 0.75])
[['Bb'], 0.75, ['Eb'], 0.75, ['Eb'], 0.5, ['D'], 0.5, ['Gb'], 0.75]
[['Db'], 0.5, ['A'], 0.5, ['F'], 0.75, ['F'], 0.75, ['G'], 0.5]
[['E'], 0.75, ['Gb'], 0.5, ['Gb'], 0.5, ['C'], 0.25, ['Gb'], 0.5]
[['Eb'], 0.25, ['Eb'], 0.75, ['Bb'], 0.25, ['G'], 0.75, ['Eb'], 0.5]
[['A'], 0.75, ['Bb'], 0.75, ['A'], 0.5, ['A'], 0.5, ['D'], 0.5]
